In [1]:
from collections import defaultdict

puzzle = defaultdict(int)

with open("inputs/day17-input.txt") as f:
    for key, value in enumerate(map(int, f.read().split(","))):
        puzzle[key] = value

## Part 1

In [2]:
%%time

from typing import Dict, List

class IntcodeComputer:
    def __init__(self, intcode: Dict[int, int]):
        self.intcode = intcode
        self.input = []

        
    def get_param_index(self, instruction_index: int, parameter: int) -> int:
        instruction = f"{self.intcode[instruction_index]:05d}"
        mode = instruction[-2 - parameter]
        is_position  = (mode == "0")
        is_immediate = (mode == "1")
        is_relative  = (mode == "2")
        
        param_index = instruction_index + parameter
        param_value = self.intcode[param_index]
        
        if is_position:
            return param_value
        
        elif is_immediate:
            return param_index
        
        elif is_relative:
            return self.relative_base + param_value
        
    def get_param(self, instruction_index: int, parameter: int) -> int:
        return self.intcode[self.get_param_index(instruction_index, parameter)]


    def get_opcode(self, instruction_index: int) -> int:
        opcode = str(self.intcode[instruction_index])[-2:]
        return int(opcode)


    def run_program(self) -> int:
        index = 0
        self.relative_base = 0
        
        while True:
            opcode = self.get_opcode(index)

            if opcode == 99:
                yield True
                break

            if opcode == 1:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 + in2
                index += 4

            elif opcode == 2:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 * in2
                index += 4

            elif opcode == 3:
                in1 = self.get_param_index(index, 1)

                self.intcode[in1] = self.input.pop(0)
                index += 2

            elif opcode == 4:
                in1 = self.get_param(index, 1)

                self.result = in1
                yield False
                index += 2

            elif opcode == 5:
                in1 = self.get_param(index, 1)
                if in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 6:
                in1 = self.get_param(index, 1)
                if not in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 7:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 < in2)
                index += 4

            elif opcode == 8:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 == in2)
                index += 4
                
            elif opcode == 9:
                in1 = self.get_param(index, 1)
                
                self.relative_base += in1
                index += 2
    

def print_view(view):
    max_x = max(view, key=lambda x: x[0])[0]
    max_y = max(view, key=lambda x: x[1])[1]
    
    for y in range(max_y + 1):
        for x in range(max_x + 1):
            print(view[(x, y)], end="")
        print()
    
computer = IntcodeComputer(puzzle.copy())
program = computer.run_program()

view = {}
position = (0, 0)
halted = False

while not halted:
    halted = next(program)
    
    if computer.result == 10:
        position = (-1, position[1] + 1)
    else:
        view[position] = chr(computer.result)
        
    position = (position[0] + 1, position[1])
    


print_view(view)

directions = [(0, 0), (0, -1), (1, 0), (0, 1), (-1, 0)]

def move(position, direction):
    return (position[0] + direction[0], position[1] + direction[1])

def is_intersection(view, position):
    return all(map(lambda x: view.get(move(position, x), "") == "#", directions))

intersections = list(filter(lambda x: is_intersection(view, x), view))
print(sum(map(lambda x: x[0] * x[1], intersections)))

..............................................#######................................
..............................................#......................................
..............................................#......................................
..............................................#......................................
..............................................#......................................
..............................................#......................................
..........................................#############..............................
..........................................#...#.......#..............................
..........................................#...#.......#.........###########..........
..........................................#...#.......#.........#.........#..........
..........................................#...#.......#.........#.........#..........
..........................................#...#.......

## Part 2

In [3]:
%%time

from itertools import combinations
import time

vacuum_directions = {
    "^": (0, -1),
    "v": (0, 1),
    "<": (-1, 0),
    ">": (1, 0)
}

indexed_directions = {
    0: (0, -1),
    1: (1, 0),
    2: (0, 1),
    3: (-1, 0)
}

def left_right(view, position, direction):
    index = next(filter(lambda x: x[1] == direction, indexed_directions.items()))[0]
    
    right = indexed_directions[(index + 1) % 4]
    left  = indexed_directions[(index - 1) % 4]
    if view.get(move(position, right), "") == "#":
        return ("R", right)
    elif view.get(move(position, left), "") == "#":
        return ("L", left)
    else:
        return False

def find_path(view):
    position = next(filter(lambda x: view[x] in "^v<>", view))
    direction = vacuum_directions[view[position]]
    steps = 0
    path = []
    
    while True:
        if view.get(move(position, direction), "") != "#":
            turn = left_right(view, position, direction)
            if steps > 0:
                path.append(str(steps))
                steps = 0
            
            if not turn:
                break
            
            path.append(turn[0])
            direction = turn[1]
            
        position = move(position, direction)
        steps += 1
        
    return ",".join(path)


def find_subpaths(path):
    names = list("ABC")
    functions = {}
    start = 0
    starts = []
    
    while path.find(",", start) != -1:
        starts.append(start + 1 if start else 0)
        start = path.find(",", start + 1)
        start = path.find(",", start + 1)
        
    possible_functions = []
    for start in starts:
        for end in filter(lambda x: 0 < x - start <= 20, starts + [len(path) + 1]):
            possible_functions.append(path[start: end-1])
    
    for functions in combinations(possible_functions, 3):
        test_path = path [:]
        
        for name, function in zip(names, functions):
            test_path = test_path.replace(function, name)
            
        if all(x in "ABC," for x in test_path):
            break
    
    return (test_path, functions)


path = find_path(view)
main, functions = find_subpaths(path)

intcode = puzzle.copy()
intcode[0] = 2
computer = IntcodeComputer(intcode)

computer.input.extend(map(ord, main + "\n"))
for function in functions:
    computer.input.extend(map(ord, function + "\n"))
computer.input.extend(map(ord, "n\n"))

program = computer.run_program()
while not next(program):
    pass

print(computer.result)

1681189
CPU times: user 882 ms, sys: 5.24 ms, total: 887 ms
Wall time: 920 ms
